In [1]:
pip install gym

In [0]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1

In [4]:
!apt-get install x11-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils
0 upgraded, 2 newly installed, 0 to remove and 7 not upgraded.
Need to get 209 kB of archives.
After this operation, 711 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Fetched 209 kB in 2s (136 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 148036 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf

In [0]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

import warnings
warnings.filterwarnings('ignore')

In [0]:
from pyvirtualdisplay import Display
#display = Display(visible=0, size=(400, 300))
#display.start()

env = gym.make("MsPacman-v0")
env.reset()
#prev_screen = env.render(mode='rgb_array')
#plt.imshow(prev_screen)

steps = 0;
done = False
while not done:
  steps+=1
  action = env.action_space.sample()
  obs, reward, done, info = env.step(action)
  #screen = env.render(mode='rgb_array')
  
  #plt.imshow(screen)
  #ipythondisplay.clear_output(wait=True)
  #ipythondisplay.display(plt.gcf())

    
#ipythondisplay.clear_output(wait=True)
env.close()



In [7]:
print("Observations: ")
print(obs)
print("Reward: ")
print(reward)
print("Steps: ")
print(steps)
print("Information: ")
print(info)


Observations: 
[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[228 111 111]
  [228 111 111]
  [228 111 111]
  ...
  [228 111 111]
  [228 111 111]
  [228 111 111]]

 [[228 111 111]
  [228 111 111]
  [228 111 111]
  ...
  [228 111 111]
  [228 111 111]
  [228 111 111]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]
Reward: 
0.0
Steps: 
601
Information: 
{'ale.lives': 0}


In [0]:
#display.stop()

In [0]:
from statistics import median, mean
from collections import Counter
import random

goal_steps = 500
score_requirement = 50
initial_games = 10

def initial_population():
    # [OBS, MOVES]
    training_data = []
    # all scores:
    scores = []
    # just the scores that met our threshold:
    accepted_scores = []
    # iterate through however many games we want:
    for _ in range(initial_games):
        score = 0
        # moves specifically from this environment:
        game_memory = []
        # previous observation that we saw
        prev_observation = []
        # for each frame in 200
        for _ in range(goal_steps):
            # choose random action (0 or 1)
            action = random.randrange(0,4)
            # do it!
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done: break

        # IF our score is higher than our threshold, we'd like to save
        # every move we made
        # NOTE the reinforcement methodology here. 
        # all we're doing is reinforcing the score, we're not trying 
        # to influence the machine in any way as to HOW that score is 
        # reached.
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                # convert to one-hot (this is the output layer for our neural network)
                if data[1] == 0:
                    output = [1,0,0,0]
                elif data[1] == 1:
                    output = [0,1,0,0]
                elif data[1] == 2:
                    output = [0,0,1,0]
                elif data[1] == 3:
                    output = [0,0,0,1]
                    
                # saving our training data
                training_data.append([data[0], output])

        # reset env to play again
        env.reset()
        # save overall scores
        scores.append(score)
    
    # just in case you wanted to reference later
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    # some stats here, to further illustrate the neural network magic!
    print('Average accepted score:',mean(accepted_scores))
    print('Median score for accepted scores:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [21]:
tr_data = initial_population()

Average accepted score: 130.0
Median score for accepted scores: 130.0
Counter({130.0: 1})


In [0]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

LR = 1e-3

def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 4, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 4, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 4, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 4, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 4, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 4, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')

    return model


def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    print('1')
    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    print('2')
    model.fit({'input': X}, {'targets': y}, n_epoch=2, snapshot_step=200, show_metric=True, run_id='openai_learning')
    print('3')
    return model

In [0]:
model = train_model(tr_data)

1
2
---------------------------------
Run id: openai_learning
Log directory: log/
INFO:tensorflow:Summary name Accuracy_6/Adam_0 (raw) is illegal; using Accuracy_6/Adam_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_7/Adam_1 (raw) is illegal; using Accuracy_7/Adam_1__raw_ instead.


Exception in thread Thread-69:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tflearn/data_flow.py", line 187, in fill_feed_dict_queue
    data = self.retrieve_data(batch_ids)
  File "/usr/local/lib/python3.6/dist-packages/tflearn/data_flow.py", line 222, in retrieve_data
    utils.slice_array(self.feed_dict[key], batch_ids)
  File "/usr/local/lib/python3.6/dist-packages/tflearn/utils.py", line 187, in slice_array
    return X[start]
IndexError: index 47372 is out of bounds for axis 0 with size 459

Exception in thread Thread-71:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/u

---------------------------------
Training samples: 1762560
Validation samples: 0
--


In [0]:
!mkdir shen
!ls